In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from scipy.stats import entropy
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt

# Load and preprocess data
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# Split training data into training and validation sets
X_train, X_validation, y_train, y_validation = train_test_split(
    X_train_full, y_train_full, test_size=0.2, random_state=2020
)

# Expand dimensions for channels
X_train = np.expand_dims(X_train, -1)
X_validation = np.expand_dims(X_validation, -1)
X_test = np.expand_dims(X_test, -1)

# Define the CNN model
def create_model():
    model = keras.Sequential([
        layers.Conv2D(32, kernel_size=3, activation='relu', input_shape=(28, 28, 1)),
        layers.MaxPooling2D(pool_size=2),
        layers.Conv2D(64, kernel_size=3, activation='relu'),
        layers.MaxPooling2D(pool_size=2),
        layers.Flatten(),
        layers.Dense(128, activation='relu'),
        layers.Dropout(0.25),
        layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model

# Metrics Calculation
def calculate_metrics(model, X_pool):
    predictions = model.predict(X_pool, batch_size=256)
    features = model.predict(X_pool, batch_size=512)

    # Least Confidence
    least_conf_scores = np.max(predictions, axis=1)
    avg_least_confidence = np.mean(least_conf_scores)

    # Prediction Entropy
    pred_entropies = entropy(predictions.T)
    avg_prediction_entropy = np.mean(pred_entropies)

    # Margin Sampling
    sorted_preds = -np.sort(-predictions, axis=1)
    margins = sorted_preds[:, 0] - sorted_preds[:, 1]
    avg_margin_sampling = np.mean(margins)

    # Cosine Similarity (Feature Diversity)
    similarities = cosine_similarity(features)
    diversities = 1 - similarities.sum(axis=1)
    avg_cosine_similarity = np.mean(diversities)

    # L2 Norm
    l2_norms = np.linalg.norm(features, axis=1)
    avg_l2_norm = np.mean(l2_norms)

    # KL Divergence
    uniform_dist = np.ones_like(predictions) / predictions.shape[1]
    kl_divergences = np.sum(predictions * np.log(predictions / uniform_dist), axis=1)
    avg_kl_divergence = np.mean(kl_divergences)

    return {
        "Average Least Confidence": avg_least_confidence,
        "Average Prediction Entropy": avg_prediction_entropy,
        "Average Margin Sampling": avg_margin_sampling,
        "Average Cosine Similarity": avg_cosine_similarity,
        "Average L2 Norm": avg_l2_norm,
        "Average KL Divergence": avg_kl_divergence
    }

# Train model without active learning
model_without_al = create_model()
history_without_al = model_without_al.fit(
    X_train, y_train, epochs=20, batch_size=512, validation_data=(X_validation, y_validation)
)
val_acc_without_al = model_without_al.evaluate(X_validation, y_validation, verbose=0)[1]
print(f'Validation Accuracy without Active Learning: {val_acc_without_al:.2f}')

# Active learning selection
n_samples = 1000
selected_indices = np.random.choice(X_validation.shape[0], n_samples, replace=False)  # Random selection
X_selected = X_validation[selected_indices]
y_selected = y_validation[selected_indices]

# Augment training data with selected samples
X_train_al = np.concatenate([X_train, X_selected], axis=0)
y_train_al = np.concatenate([y_train, y_selected], axis=0)

# Train model with active learning
model_with_al = create_model()
history_with_al = model_with_al.fit(
    X_train_al, y_train_al, epochs=25, batch_size=256, validation_data=(X_validation, y_validation)
)
val_acc_with_al = model_with_al.evaluate(X_validation, y_validation, verbose=0)[1]
print(f'Validation Accuracy with Active Learning: {val_acc_with_al:.2f}')

# Calculate metrics for the validation pool
metrics_before_al = calculate_metrics(model_without_al, X_validation)
metrics_after_al = calculate_metrics(model_with_al, X_validation)

# Print metrics
print("\nMetrics Before Active Learning:")
for metric, value in metrics_before_al.items():
    print(f"{metric}: {value:.4f}")

print("\nMetrics After Active Learning:")
for metric, value in metrics_after_al.items():
    print(f"{metric}: {value:.4f}")

# Plot accuracy graphs
plt.figure(figsize=(10, 6))
plt.plot(history_without_al.history['val_accuracy'], label='Without Active Learning', linestyle='--')
plt.plot(history_with_al.history['val_accuracy'], label='With Active Learning', linestyle='-')
plt.xlabel('Epochs')
plt.ylabel('Validation Accuracy')
plt.title('Validation Accuracy With and Without Active Learning')
plt.legend()
plt.grid(True)
plt.show()

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
86/94 ━━━━━━━━━━━━━━━━━━━━ 3s 477ms/step - accuracy: 0.5532 - loss: 1.2983